<a href="https://colab.research.google.com/github/MShirazAhmad/toolbox/blob/main/python/nanoindentation/Nanoindentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Run this code block to download sample file named "Silica.xls"

!wget https://raw.githubusercontent.com/MShirazAhmad/toolbox/refs/heads/main/python/nanoindentation/Silica.xls

# Write xls name below


In [ ]:
filename = "Silica.xls"

# Calculating Offsets based on loading curve

In [ ]:
#@title Step 1: Run this codeblock



"""nano_offset_plotter_refactored.py — Compute x-offsets for nano-indentation curves (dual backend)
==============================================================================================
* Moving-average smoothing of the *Load* signal (default 200 pts).
* Quadratic fit of the smoothed loading branch with optional x-cutoff (nm).
* Plotting backends: interactive **Plotly** (default) or static **Matplotlib** via `backend` argument.
* Function API (`collect_offsets`) and CLI.
"""
from __future__ import annotations

import argparse
from typing import Optional, Dict, List, Tuple

import numpy as np
import pandas as pd
import csv
import os
# Plotting libs
import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as mpl

# Defaults
DEFAULT_WINDOW = 0         # moving-average window (pts)
DEFAULT_CUTOFF = 0         # nm (≤0 ⇒ full branch)
DEFAULT_BACKEND = "plotly" # or "mpl"
PLOTLY_TEMPLATE = "plotly_white"

# Helpers
def moving_average(a: np.ndarray, window: int = DEFAULT_WINDOW) -> np.ndarray:
    if window < 2 or len(a) < window:
        return a
    return np.convolve(a, np.ones(window) / window, mode="same")

def compute_x_offset(
    x: np.ndarray,
    y: np.ndarray,
    *,
    max_x: Optional[float] = DEFAULT_CUTOFF,
    window: int = DEFAULT_WINDOW
) -> Tuple[float, str]:
    # Identify loading branch (up to peak load)
    peak_idx = np.argmax(y)
    x_load = x[:peak_idx + 1]
    y_load = y[:peak_idx + 1]

    # Apply cutoff if specified
    mask = (x_load <= max_x) if (max_x and max_x > 0) else np.ones_like(x_load, bool)
    x_fit = x_load[mask]
    y_fit = moving_average(y_load, window)[mask]

    # 1. Fit cubic polynomial
    coeffs = np.polyfit(x_fit, y_fit, 2)
    # 2. Evaluate the polynomial at the data points
    y_pred = np.polyval(coeffs, x_fit)

    # 3. Compute residual and total sums of squares
    ss_res = np.sum((y_fit - y_pred) ** 2)
    ss_tot = np.sum((y_fit - np.mean(y_fit)) ** 2)

    # 4. Coefficient of determination (R²)
    r_squared = 1 - ss_res / ss_tot
    # 5. Pearson correlation coefficient (r)
    r = np.corrcoef(y_fit, y_pred)[0, 1]

    r2_str = f"R²={r_squared:.4f}, r={r:.4f}"
    print(r2_str)

    # 6. Roots → real → within range → choose closest to zero
    roots = np.roots(coeffs)
    real_roots = roots[np.isreal(roots)].real
    xmin, xmax = x_fit.min(), x_fit.max()
    valid_roots = [rt for rt in real_roots if xmin <= rt <= xmax]
    x_intercept = float(valid_roots[0] if valid_roots else min(real_roots, key=abs))

    return x_intercept, r2_str

def load_test_data(fname: str, tno: str) -> pd.DataFrame:
    df = pd.read_excel(fname, sheet_name=f"Test {tno}").drop(index=[0, 1])
    df["Displacement Into Surface"] = pd.to_numeric(df["Displacement Into Surface"], errors="coerce")
    df["Load On Sample"]           = pd.to_numeric(df["Load On Sample"], errors="coerce")
    return df.dropna(subset=["Displacement Into Surface", "Load On Sample"])

# Unified plotting function
def plot_curves(
    dataset: List[Tuple[pd.DataFrame, str, float, str]],
    *,
    window: int,
    cutoff: Optional[float],
    backend: str = DEFAULT_BACKEND,
    plot_mode: str = "separate",
    fname: str
) -> None:
    """
    Plot nanoindentation curves using Plotly or Matplotlib, in separate or combined mode.

    Parameters
    ----------
    dataset    : List of (DataFrame, test_number, x_offset, r2_str) tuples
    window     : Moving-average window length (points)
    cutoff     : Fit only x ≤ cutoff nm; <=0 or None → full curve
    backend    : 'plotly' or 'mpl'
    plot_mode  : 'separate' or 'combined'
    fname      : source Excel filename (for HTML export)
    """
    if not dataset:
        return

    fit_tag = "full" if not cutoff or cutoff <= 0 else f"≤{cutoff:g} nm"
    fig = go.Figure()
    palette = px.colors.qualitative.Plotly * 6

    for idx, (df, tno, x_off, r2_str) in enumerate(dataset):
        df = df.copy()
        df["Displacement Into Surface"] -= x_off
        peak = df["Load On Sample"].idxmax()
        dfL, dfU = df.iloc[:peak+1], df.iloc[peak+1:]

        # Cutoff-aware fit on loading branch
        if cutoff and cutoff > 0:
            maskL = dfL["Displacement Into Surface"] <= cutoff
            x_plot_max = cutoff
        else:
            maskL = np.ones(len(dfL), bool)
            x_plot_max = dfL["Displacement Into Surface"].max()

        # Plot raw data
        fig.add_trace(go.Scatter(
            x=dfL["Displacement Into Surface"],
            y=dfL["Load On Sample"],
            mode="markers",
            marker=dict(symbol="circle-open", color=palette[idx], opacity=0.2),
            name=f"Test {tno} ({r2_str})"
        ))
        fig.add_trace(go.Scatter(
            x=dfU["Displacement Into Surface"],
            y=dfU["Load On Sample"],
            mode="markers",
            marker=dict(symbol="circle-open", color=palette[idx], opacity=0.2),
            showlegend=False
        ))

        # Fitted curve
        x_fitL = dfL["Displacement Into Surface"][maskL]
        y_fitL = moving_average(dfL["Load On Sample"], window)[maskL]
        cL = np.polyfit(x_fitL, y_fitL, 2)
        xL = np.linspace(0, x_plot_max, 200)
        fig.add_trace(go.Scatter(
            x=xL,
            y=np.polyval(cL, xL),
            mode="lines",
            line=dict(color=palette[idx]),
            showlegend=False
        ))

        fig.update_layout(
            template=PLOTLY_TEMPLATE,
            # title=f"Combined plots ({window}-pt avg, fit: {fit_tag})",
            xaxis=dict(range=[0, None], title="Displacement / nm", tickformat="02d"),
            yaxis=dict(range=[0, None], title="Load / mN"),
            legend=dict(
                orientation="h",      # horizontal legend
                yanchor="bottom",     # anchor legend at its bottom
                y=1.02,               # place it just above the top of the plot
                xanchor="center",     # anchor legend at its center
                x=0.5                 # center the legend horizontally
            )
        )


    fig.show()

    # Save interactive HTML
    base_name = os.path.splitext(os.path.basename(fname))[0]
    html_filename = f"Offsets_{base_name}.html"
    fig.write_html(html_filename, include_plotlyjs="cdn")
    print(f"Saved interactive plot to {html_filename}")

def collect_offsets(
    fname: str,
    *,
    first: int = 1,
    last: int = 1,
    plot_mode: str = "separate",
    backend: str = "plotly",
    window: int = 50,
    cutoff_nm: Optional[float] = DEFAULT_CUTOFF,
) -> Dict[str, Tuple[float, str]]:
    """
    Batch-process Test sheets and return a dict of (offset, R² string).
    """
    offsets: Dict[str, Tuple[float, str]] = {}
    combined: List[Tuple[pd.DataFrame, str, float, str]] = []

    for n in range(first, last + 1):
        tno = f"{n:03d}"
        try:
            df = load_test_data(fname, tno)
            off, r2_str = compute_x_offset(
                df["Displacement Into Surface"].to_numpy(),
                df["Load On Sample"].to_numpy(),
                max_x=cutoff_nm,
                window=window,
            )
            offsets[tno] = (off, r2_str)
            print(f"Test {tno}: x_offset={off:.3f} nm, {r2_str}")

            if plot_mode in ("separate", "combined"):
                combined.append((df, tno, off, r2_str))
        except Exception as e:
            print(f"Test {tno}: {e}")

    if plot_mode in ("separate", "combined") and combined:
        plot_curves(
            combined,
            window=window,
            cutoff=cutoff_nm,
            backend=backend,
            plot_mode=plot_mode,
            fname=fname
        )

    return offsets

def save_offsets(offsets: Dict[str, Tuple[float, str]], filename: str) -> None:
    """Save offsets and R² to a CSV file"""
    csv_fn = f"{filename}_offsets.csv"
    with open(csv_fn, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(["Test", "x_offset_nm", "R_squared"])
        for tno, (off, r2_str) in offsets.items():
            writer.writerow([tno, f"{off:.6f}", r2_str])
    print(f"Saved offsets and R² to {csv_fn}")


# save_offsets(offsets, filename)


In [ ]:
#@title Run this codeblock to generate and save offsets csv

offsets = collect_offsets(
    filename,
    first=1,
    last=40,
    window=20,
    cutoff_nm=1000,
)
save_offsets(offsets, filename)

# Generating Graph for Manuscript

In [ ]:
#@title Step 1: Run this code block

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# --- Global Styling ---
plt.rcParams.update({
    "figure.figsize": (8, 6),
    "font.size": 14,
    "axes.linewidth": 1.5,
    "lines.linewidth": 2.5,
    "font.family": "sans-serif",
    "axes.prop_cycle": plt.cycler(color=[
        "#1f77b4", "#ff7f0e", "#2ca02c", "#d62728",
        "#9467bd", "#8c564b", "#e377c2", "#7f7f7f",
        "#bcbd22", "#17becf"
    ]),
    "path.simplify_threshold": 0.0,
    "agg.path.chunksize": 10000,
})

def read_combined_excel(filename, test_numbers, x_pattern, hardness_pattern):
    combined_df = pd.DataFrame()
    for i in test_numbers:
        sheet = f"Test {str(i).zfill(3)}"
        try:
            df = pd.read_excel(filename, sheet_name=sheet, dtype=str, engine='xlrd')
            df.columns = [
                'Segment',
                f'{x_pattern} - {i}',
                f'Load On Sample (mN) - {i}',
                f'Time On Sample (s) - {i}',
                f'Harmonic Contact Stiffness (N/m) - {i}',
                f'{hardness_pattern} - {i}',
                f'Modulus (GPa) - {i}'
            ]
            df = df.drop(columns=['Segment'])
            combined_df = pd.concat([combined_df, df], axis=1) if not combined_df.empty else df
        except Exception as e:
            print(f"Error reading sheet '{sheet}': {e}")
    return combined_df

# --- Stats with Inf/NaN Filtering ---
def calculate_mean_std_count(df, patterns):
    cols = [c for c in df.columns if any(p in c for p in patterns)]
    numeric = df[cols].apply(pd.to_numeric, errors='coerce')
    mean = numeric.mean(axis=1)
    std  = numeric.std(axis=1, ddof=1)
    cnt  = numeric.count(axis=1)
    mean.replace([np.inf, -np.inf], np.nan, inplace=True)
    std.replace([np.inf, -np.inf], np.nan, inplace=True)
    return mean, std, cnt

# --- Weighted Propagation ---
def propagate_weighted_average_and_uncertainty(vals, sds, cnts):
    vals = np.asarray(vals, dtype=np.float64)
    sds  = np.asarray(sds, dtype=np.float64)
    cnts = np.asarray(cnts, dtype=np.float64)

    # Filter only rows with finite values
    mask = np.isfinite(vals) & np.isfinite(sds) & np.isfinite(cnts)
    vals, sds, cnts = vals[mask], sds[mask], cnts[mask]

    if len(vals) == 0 or cnts.sum() == 0:
        return np.nan, np.nan

    # --- Force-safe values ---
    sds = np.clip(sds, 0, 100)           # Clamp to avoid overflow
    cnts = np.clip(cnts, 1e-3, 1e5)      # Avoid zero-division
    w = cnts / cnts.sum()
    w = np.clip(w, 0, 1)

    # --- Compute mean ---
    mean = np.dot(w, vals)

    # --- Compute variance safely ---
    safe_var_terms = (w**2) * (sds**2)
    if np.any(~np.isfinite(safe_var_terms)):
        safe_var_terms = np.nan_to_num(safe_var_terms, nan=0.0, posinf=0.0, neginf=0.0)

    var = np.sum(safe_var_terms)
    return mean, np.sqrt(var)



# --- Plot Helper ---
def plot_curve_with_errors(ax, x, y, yerr, idx, color, label, marker):
    mask = np.isfinite(x) & np.isfinite(y) & np.isfinite(yerr)
    x_f, y_f, e_f = map(lambda s: pd.Series(s, dtype=np.float64), (x[mask], y[mask], yerr[mask]))
    ax.plot(x_f, y_f, color=color, label=label, linewidth=2.5)
    if isinstance(idx, (list, np.ndarray)):
        for i in idx:
            if i < len(x_f):
                ax.errorbar(x_f.iloc[i], y_f.iloc[i], yerr=e_f.iloc[i], fmt=marker,
                            markersize=6, linestyle='none', capsize=3,
                            color=color, linewidth=1.5)
    else:
        if idx < len(x_f):
            ax.errorbar(x_f.iloc[idx], y_f.iloc[idx], yerr=e_f.iloc[idx], fmt=marker,
                        markersize=6, linestyle='none', capsize=3,
                        color=color, linewidth=1.5)

# --- Main Comparison ---
def compare_files_with_weighted_uncertainty(
    file_configs,
    x_pattern,
    hardness_pattern,
    modulus_pattern,
    target_x_range,
    hardness_ylim,
    modulus_ylim
):
    x_min, x_max, step = target_x_range
    h_min, h_max = hardness_ylim
    m_min, m_max = modulus_ylim

    fig, (ax1, ax2) = plt.subplots(
        2, 1, figsize=(8, 6), sharex=True,
        gridspec_kw={'hspace': 0.05}, constrained_layout=True
    )

    for cfg in file_configs:
        df = read_combined_excel(cfg['filename'], cfg['test_numbers'], x_pattern, hardness_pattern)
        if df.empty:
            continue
        x_col = next((c for c in df.columns if x_pattern in c), None)
        x = pd.to_numeric(df[x_col], errors='coerce')

        m_h, s_h, c_h = calculate_mean_std_count(df, [hardness_pattern])
        m_m, s_m, c_m = calculate_mean_std_count(df, [modulus_pattern])

        mask_h = (x >= x_min) & (x <= x_max) & (m_h >= h_min) & (m_h <= h_max)
        mask_m = (x >= x_min) & (x <= x_max) & (m_m >= m_min) & (m_m <= m_max)

        # Hardness
        if mask_h.any():
            idxs_h = [abs(x[mask_h] - v).argmin() for v in np.arange(x_min, x_max, step)]
            plot_curve_with_errors(ax1, x[mask_h], m_h[mask_h], s_h[mask_h], idxs_h,
                                   cfg['hardness_color'], cfg['label'], 'o')
            avg_h, err_h = propagate_weighted_average_and_uncertainty(m_h[mask_h], s_h[mask_h], c_h[mask_h])
            print(f"{cfg['label']} - Weighted Mean Hardness: {avg_h:.2f} ± {err_h:.2f} GPa")
        else:
            print(f"{cfg['label']} - No hardness data within specified limits, skipping.")

        # Modulus
        if mask_m.any():
            idxs_m = [abs(x[mask_m] - v).argmin() for v in np.arange(x_min, x_max, step)]
            plot_curve_with_errors(ax2, x[mask_m], m_m[mask_m], s_m[mask_m], idxs_m,
                                   cfg['modulus_color'], cfg['label'], 's')
            avg_m, err_m = propagate_weighted_average_and_uncertainty(m_m[mask_m], s_m[mask_m], c_m[mask_m])
            print(f"{cfg['label']} - Weighted Mean Modulus: {avg_m:.2f} ± {err_m:.2f} GPa\n")
        else:
            print(f"{cfg['label']} - No modulus data within specified limits, skipping.\n")

    for ax in (ax1, ax2):
        ax.minorticks_on()
        ax.grid(True, which='both', linestyle='--', linewidth=0.5)

    ax1.set_xlim(x_min, x_max)
    ax1.set_ylim(h_min, h_max)
    ax1.set_ylabel(hardness_pattern)

    ax2.set_xlim(x_min, x_max)
    ax2.set_ylim(m_min, m_max)
    ax2.set_xlabel(x_pattern)
    ax2.set_ylabel(modulus_pattern)

    # Legend guard
    handles, labels = ax1.get_legend_handles_labels()
    if labels:
        fig.legend(handles, labels, loc='upper center', bbox_to_anchor=(0.5, 0.99),
                   ncol=len(labels), frameon=False)

    fig.savefig("comparison_with_weighted_uncertainty.png", dpi=300)
    plt.show()


In [ ]:
#title Graph Configs

file_configs_single = [
    {
        'filename': filename,
        'test_numbers': [1, 2, 3, 4, 5],  # All tests for this sample
        'label': 'My Material Sample',
        'hardness_color': '#1f77b4',  # Blue
        'modulus_color': '#1f77b4',   # Blue
        'offset_csv': 'Silica.xls_offsets.csv'  # Optional: correct depth offsets
    }
]

# Run the comparison for single sample
compare_files_with_weighted_uncertainty(
    file_configs=file_configs_single,
    x_pattern='Depth (nm)',           # Column pattern for x-axis
    hardness_pattern='Hardness (GPa)', # Column pattern for hardness
    modulus_pattern='Modulus (GPa)',   # Column pattern for modulus
    target_x_range=(100, 200, 50),     # (x_min, x_max, step) in nm
    hardness_ylim=(0, 30),              # (min, max) for hardness y-axis
    modulus_ylim=(0, 200)            # (min, max) for modulus y-axis
)